In [1]:
from pettingzoo.mpe import simple_spread_v3
import time
import argparse


In [2]:
import argparse
import time
import datetime
import numpy as np 
import torch
import os
import sys
import itertools
from copy import deepcopy
from buffer import ReplayMemory
from train import AgentsTrainer
max_taining_steps = 600
start_steps = max_taining_steps//5
parser = argparse.ArgumentParser(description='PyTorch QMIX Args')
parser.add_argument('--scenario', type=str, default='simple_spread', help='name of the scenario script')
parser.add_argument('--num_episodes', type=int, default=max_taining_steps, help='number of episodes for training')
parser.add_argument('--max_episode_len', type=int, default=25, help='maximum episode length')
parser.add_argument('--policy_lr', type=float, default=1e-4, help='learning rate for policies')
parser.add_argument('--critic_lr', type=float, default=1e-2, help='learning rate for critics')
parser.add_argument('--alpha', type=float, default=0.0, help='policy entropy term coefficient')
parser.add_argument('--tau', type=float, default=0.05, help='target network smoothing coefficient')
parser.add_argument('--gamma', type=float, default=0.99, help='discount factor (default: 0.99)')
parser.add_argument('--seed', type=int, default=123, help='random seed (default: 123)')
parser.add_argument('--batch_size', type=int, default=32, help='batch size (default: 16)') # episodes
parser.add_argument('--hidden_dim', type=int, default=64, help='network hidden size (default: 256)')
parser.add_argument('--start_steps', type=int, default=start_steps, help='steps before training begins')
parser.add_argument('--target_update_interval', type=int, default=1, help='tagert network update interval')
parser.add_argument('--updates_per_step', type=int, default=1, help='network update frequency')
parser.add_argument('--replay_size', type=int, default=50000, help='maximum number of episodes of replay buffer')
parser.add_argument('--cuda', action='store_false', help='run on GPU (default: False)')
parser.add_argument('--render', action='store_true', help='render or not')
sys.argv=['']
args = parser.parse_args()

np.random.seed(args.seed)
torch.manual_seed(args.seed)

# TensorboardX


# Load environment
N = 3
env = simple_spread_v3.env(N=N, local_ratio=0.5, max_cycles=args.max_episode_len, continuous_actions=True,render_mode='human')


# Homogeneity check
obs_shape = 10
action_shape = 1

memory = ReplayMemory(args.replay_size, args.max_episode_len, N, obs_shape, action_shape)

trainer = AgentsTrainer(N, obs_shape, action_shape, args)
total_numsteps = 0
updates = 0

# reward_bias = 30.
train_policy = False
for i_episode in itertools.count(1):
    episode_reward = 0.0 # sum of all agents
    step_within_episode = 0

    env.reset()
    done = False
    terminated = False

    while not done and not terminated:
        observation = np.random.normal(loc = 0, scale = 1, size=(N, obs_shape))
        action_list = np.random.normal(loc = 0, scale = 1, size=(N, action_shape))

        # interact with the environment
        # new_obs_list, reward_list, done_list, _ = env.step(np.array(action_list))
        reward = np.random.normal(loc = 0, scale = 1)
        new_observation = np.random.normal(loc = 0, scale = 1, size=(N, obs_shape))
        new_obs_list = env.state()
        done = np.random.binomial(n=1,p=0.05)
        
        total_numsteps += 1
        step_within_episode += 1
        terminated = (step_within_episode >= args.max_episode_len)

        # replay memory filling
        memory.push(observation, action_list, reward, new_observation, 1. if (done and not terminated) else 0.)
        # memory.push(np.asarray(obs_list), np.asarray(action_list), reward_list[0], np.asarray(new_obs_list),
        #              1. if done else 0.)
        obs_list = new_obs_list

        episode_reward += reward

    memory.end_episode()
    trainer.reset()

    if i_episode > 1000:
        train_policy = True
    if len(memory) > args.batch_size:
        for _ in range(args.updates_per_step):
            obs_batch, action_batch, reward_batch, next_obs_batch, mask_batch, done_batch = memory.sample(args.batch_size)
            sample_batch = (obs_batch, action_batch, reward_batch, next_obs_batch, mask_batch, done_batch)
            critic_loss, policy_loss = trainer.update_parameters(sample_batch, args.batch_size, updates, train_policy)
            cl_mean = np.mean(np.asarray(critic_loss))
            pl_mean = np.mean(np.asarray(policy_loss))
            updates += 1
    else:
        cl_mean = 0. 
        pl_mean = 0. 

    # print("Episode: {}, total steps: {}, total episodes: {}, reward: {}".format(i_episode, total_numsteps,
    #     step_within_episode, round(episode_reward, 2)))

    if i_episode > args.num_episodes:
        break

env.close()

RuntimeError: Input batch size 32 doesn't match hidden0 batch size 96